In [1]:
%load_ext Cython
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import seaborn, time
import numpy as np

In [3]:
from scipy.stats import multivariate_normal

In [24]:
mus = np.array(range(8)).reshape(2, -1)
covs = np.diag(range(8)) + .2

In [22]:
multivariate_normal.pdf(np.random.rand(2, 4), mus, covs)[np.newaxis, :].shape

(1, 2)

In [25]:
a = np.array([[1, 2], [3, 4]])
b = np.array([[5, 6]])

In [28]:
a.shape

(2, 2)

In [29]:
b.shape

(1, 2)

In [47]:
%%cython
from cython.parallel cimport prange
cimport cython
cimport numpy as cnp
import numpy as np
from libc.stdlib cimport malloc

cpdef double mysum_par(double[:] x):
    cdef int n = x.shape[0]
    cdef int i
    cdef double sums = 0
    with nogil, cython.boundscheck(False), cython.wraparound(False):
            for i in prange(n, num_threads=8, schedule='guided'):
                sums += x[i]
    return sums

cdef double _mysum_par_pointer(double *x, int n):
    cdef int i
    cdef double sums = 0
    with nogil, cython.boundscheck(False), cython.wraparound(False):
            for i in prange(n, num_threads=8, schedule='guided'):
                sums += x[i]
    return sums
cpdef double mysum_par_pointer(cnp.ndarray x):
    cdef n = x.shape[0]
    cdef double *_x = <double*> x.data
    return _mysum_par_pointer(_x, n)
    
cpdef double mysum(double[:] x):
    cdef int n = x.shape[0]
    cdef int i
    cdef double sums=  0
    for i in range(n):
        sums += x[i]
    return sums
                       

In [38]:
a = np.random.rand(10000000)

In [48]:
%timeit mysum_par(a)

9.36 ms ± 21.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [28]:
%timeit mysum(a)

2.61 ms ± 5.35 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [29]:
%timeit mysum_par_pointer(a)

9.01 ms ± 22.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [49]:
mysum(a)

5000420.009903621

In [50]:
mysum_par(a)

5000420.009903621

In [51]:
mysum_par_pointer(a)

5000420.009903621